### Описание гипотезы
Проверим два предположения:
- Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
- Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?

Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2018 году.

### Ход проверки

Импортируем нужные библиотеки

In [2]:
import pandas as pd
import numpy as np

Получаем данные из базы данных

In [3]:
events_df = pd.read_csv('events.csv')
purchase_df = pd.read_csv('purchase.csv')

Сделаем выборку пользователей, зарегистрировавшихся в 2018 году. Период анализа — с 1 января по 31 декабря 2018 года.



In [4]:
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type == 'registration')
registered = events_df[cond]['user_id'].to_list()
events_df = events_df[events_df.user_id.isin(registered)]    
events_df.start_time = pd.to_datetime(events_df.start_time, format = '%Y-%m-%dT%H:%M:%S')

Произведем преобразование типов (в датафрейме purchase_df переименуем столбей event_datetime на start_time для дальнейшего объединения с датафреймом event_df)

In [5]:
events_df['start_time'] = pd.to_datetime(events_df.start_time)
purchase_df['start_time'] = pd.to_datetime(purchase_df.event_datetime)

Перед объединением датафреймов, переименуем колонки

In [6]:
events_df = events_df.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})

Сделаем объединенный датафрейм из событий и оплат

In [7]:
total_events_df = pd.concat([events_df,purchase_df],sort=False)

Сделаем группы пользователей по уровню сложности, который был выбран для тренировок.

Сначала посмотрим как уровни сложности бывают:

In [8]:
total_events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Сформируем группы

In [9]:
users_with_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']['user_id'].unique()
users_with_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']['user_id'].unique()
users_with_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']['user_id'].unique()

Посчитаем для каждой группы показатели

In [10]:
user_groups = [
    {'easy': users_with_easy_level},
    {'medium': users_with_medium_level},
    {'hard': users_with_hard_level}
]

for group in user_groups:
    level = list(group.keys())[0]
    group_users = group[level]
    count_of_users_in_group = len(group_users)
    purchase_df_slice = purchase_df[purchase_df['user_id'].isin(group_users)]
    percent_of_purchase = purchase_df_slice['user_id'].nunique()/count_of_users_in_group
    print ('Процент оплативших пользователей, выбравших уровень сложности {}: {:.2%}'.format(level,percent_of_purchase))
    print ()
    level_choice_df = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(group_users))]
    if (level_choice_df['user_id'].value_counts().mean()) == 1:
        level_choice_df = level_choice_df[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
        purchase_df_slice_2 = purchase_df_slice[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
        merged_df = purchase_df_slice_2.merge(level_choice_df,on='user_id',how='inner')
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['level_choice_time']
        mean_time = merged_df['timedelta'].mean()
        print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))
        print ('Характеристики времени:')
        print (merged_df['timedelta'].describe())
    else:
        print ('Более 1 события выбора уровня сложности')

Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798941
Характеристики времени:
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118679
Характеристики времени:
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%     

### Выводы
Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%
Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%

Таким образом, существует прямая зависимость между выбранным уровнем сложности и процент пользователей, которые оплатили тренировки. Пользователи, выбирающие более сложные тренировки, чаще покупают платные тренировки.

Наибольшее время между событием выбора уровня сложности и моментом первой оплаты, тратят пользователи, которые выбрали уровень сложности medium. Для них среднее время составляет 3 дня 23 часа, а медианное время 3 дня 19 часов.

Чуть меньше времени тратят пользователи, выбравшие уровень сложности easy. Для них среднее время составляет 3 дня 14 часов, а медианное время 3 дня 6 часов.

Меньше всего времени на принятие решение о покупке тратят пользователя, выбравшие уровень сложности hard. Для них среднее время составляет 3 дня 7 часов, а медианное время 3 дня 13 часов. 